# House of Cards project

### Ideas :
- Topic Modeling
- Cloud of words
- Comparison to real American politic

## 1. Information Retrieval

Get the scripts of the 13 episodes of the season 1

In [51]:
# Imports
import urllib.request
import string
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import nltk.stem

#### Web scraping

In [1]:
# Main URL
s1 = urllib.request.urlopen('https://www.springfieldspringfield.co.uk/episode_scripts.php?tv-show=house-of-cards-2013&season=1').read().decode('utf-8')

In [50]:
# get the link of the script pages for all episodes of season 1

links = re.compile(r'href=\"(view_episode_scripts\.php\?tv-show=house-of-cards-2013&episode=s01e.*?)\"')
episodes_list = re.findall(links,x)
episodes_list = list(set(episodes_list))
episodes_list = ['https://www.springfieldspringfield.co.uk/{}'.format(link) for link in episodes_list]

#### Pre-processing

In [48]:
def get_content(link) :
    page = urllib.request.urlopen(link).read().decode('utf-8')
    script_reg = re.compile(r'<div class="episode_script">.*?<div class="scrolling-script-container">(.*?)<\/div>.*?<\/div>', re.DOTALL)
    script = re.search(script_reg,page).group(1)
    return script

english_stemmer = nltk.stem.SnowballStemmer('english')
stop = set(stopwords.words('english'))

def pre_processing(script) :
    script = re.sub('<.*?>', ' ', script) #remove the tags
    script = re.sub('&.*?;', ' ', script) #remove the html symbols (ex: &amp;)
    script = re.sub('['+string.punctuation+']', ' ', script) #remove ponctuation
    
    tokens = [ word for sent in sent_tokenize(script) for word in word_tokenize(sent) ] # tokenization
    tokens = [ token.lower() for token in tokens ] # lowercase
    tokens = [ token for token in tokens if token not in stop ] # stopwords )
    tokens = [ english_stemmer.stem(token) for token in tokens ] # stem
    
    return tokens

def create_doc(link) :
    script = get_content(link)
    tokens_script = pre_processing(script)
    filename = '{}.txt'.format(re.search(r'(s01e.*?)$',link).group(1))
    with open(filename, 'w') as f:
        text = ' '.join(tokens_script)
        print(text, file=f)

In [49]:
# creation of .txt file containing the tokens (pre-processed for each episode of season 1)
for episode in episodes_list :
    create_doc(episode)